# Spark RDD vs DataFrame: Performance & API Comparison

This notebook demonstrates the fundamental differences between RDDs and DataFrames using word count on the **Complete Works of Shakespeare**.

## Key Takeaways

| Aspect | RDD | DataFrame |
|--------|-----|------------|
| **Paradigm** | Imperative (HOW) | Declarative (WHAT) |
| **Optimization** | None - executes as written | Catalyst optimizer |
| **Execution** | Java object serialization | Tungsten + code generation |
| **Type Safety** | Compile-time (Scala) | Runtime (schema) |
| **Best For** | Low-level control, unstructured data | Structured/semi-structured data |

## Setup

In [ ]:
import time
import urllib.request
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    explode, split, lower, col, regexp_replace, trim, length
)

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
# Download Shakespeare text
FILEPATH = "shakespeare.txt"
URL = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"

if not Path(FILEPATH).exists():
    print(f"Downloading from {URL}...")
    urllib.request.urlretrieve(URL, FILEPATH)

file_size = Path(FILEPATH).stat().st_size
print(f"File size: {file_size / 1024:.1f} KB ({file_size / 1024 / 1024:.2f} MB)")

In [ ]:
# Create Spark session
spark = SparkSession.builder \
    .appName("RDD_vs_DataFrame_Demo") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"Running on: {sc.master}")

---
## RDD Implementation

With RDDs, you specify **HOW** to process data step by step. Spark executes exactly what you write with no optimization.

In [ ]:
def word_count_rdd(filepath: str):
    """
    Classic RDD word count - imperative style.
    
    Step by step:
    1. textFile()     -> Read lines from file
    2. flatMap()      -> Split lines into words (one row per word)
    3. map()          -> Normalize each word
    4. filter()       -> Remove empty strings
    5. map()          -> Create (word, 1) pairs
    6. reduceByKey()  -> Sum counts per word
    7. sortBy()       -> Order by count
    """
    return (
        sc.textFile(filepath)
        .flatMap(lambda line: line.split())
        .map(lambda word: ''.join(c for c in word.lower() if c.isalpha()))
        .filter(lambda word: len(word) > 0)
        .map(lambda word: (word, 1))
        .reduceByKey(lambda a, b: a + b)
        .sortBy(lambda x: x[1], ascending=False)
    )

In [ ]:
# Execute and time
start = time.perf_counter()
rdd_result = word_count_rdd(FILEPATH).collect()
rdd_time = time.perf_counter() - start

print(f"RDD execution time: {rdd_time:.3f}s")
print(f"\nTop 10 words:")
for word, count in rdd_result[:10]:
    print(f"  {word:15} {count:,}")

---
## DataFrame Implementation

With DataFrames, you specify **WHAT** you want. The Catalyst optimizer decides **HOW** to execute efficiently.

In [ ]:
def word_count_dataframe(filepath: str):
    """
    DataFrame word count - declarative style.
    
    Benefits:
    - Catalyst optimizer rewrites query for efficiency
    - Tungsten engine: off-heap memory, code generation
    - Automatic predicate pushdown, column pruning
    """
    return (
        spark.read.text(filepath)
        # Split into words and explode to rows
        .select(explode(split(col("value"), r"\s+")).alias("word"))
        # Normalize: lowercase, keep only letters
        .select(regexp_replace(lower(col("word")), r"[^a-z]", "").alias("word"))
        # Filter empty
        .filter(length(col("word")) > 0)
        # Group and count
        .groupBy("word").count()
        # Sort
        .orderBy(col("count").desc())
    )

In [ ]:
# Execute and time
start = time.perf_counter()
df_result = word_count_dataframe(FILEPATH).collect()
df_time = time.perf_counter() - start

print(f"DataFrame execution time: {df_time:.3f}s")
print(f"\nTop 10 words:")
for row in df_result[:10]:
    print(f"  {row['word']:15} {row['count']:,}")

---
## 🔍 The Key Insight: Query Execution Plans

This is where the real difference becomes visible. The DataFrame's `explain()` shows how Catalyst transforms your query through multiple stages of optimization.

In [ ]:
# Create query (lazy - not executed yet)
df_query = word_count_dataframe(FILEPATH)

# Show the full execution plan
print("=" * 70)
print("DATAFRAME EXECUTION PLAN")
print("=" * 70)
df_query.explain(mode="formatted")

### Understanding the Plan

The plan shows several optimization stages:

1. **Parsed Logical Plan** - What you wrote
2. **Analyzed Logical Plan** - Resolved references and types
3. **Optimized Logical Plan** - After Catalyst optimization
4. **Physical Plan** - Actual execution strategy

Key optimizations you'll see:
- **WholeStageCodegen** - Compiles query stages to optimized Java bytecode
- **HashAggregate** - Efficient partial aggregation before shuffle
- **Exchange** - Shuffle operation (note: only where necessary)

In [ ]:
# RDDs have no equivalent - you can only see the DAG lineage
rdd_query = word_count_rdd(FILEPATH)

print("=" * 70)
print("RDD DEBUG STRING (Lineage only - no optimization info)")
print("=" * 70)
print(rdd_query.toDebugString().decode('utf-8'))

---
## Benchmarking: Multiple Runs

In [ ]:
def benchmark(name: str, func, n_runs: int = 5, warmup: int = 1):
    """Run benchmark with warmup."""
    # Warmup
    for _ in range(warmup):
        func()
    
    # Timed runs
    times = []
    for i in range(n_runs):
        start = time.perf_counter()
        result = func()
        times.append(time.perf_counter() - start)
    
    return {
        "name": name,
        "times": times,
        "mean": np.mean(times),
        "std": np.std(times),
        "min": np.min(times),
        "max": np.max(times)
    }

In [ ]:
N_RUNS = 5

print(f"Running {N_RUNS} benchmark iterations (plus warmup)...\n")

rdd_bench = benchmark(
    "RDD", 
    lambda: word_count_rdd(FILEPATH).collect(),
    n_runs=N_RUNS
)
print(f"RDD:       {rdd_bench['mean']:.3f}s ± {rdd_bench['std']:.3f}s")

df_bench = benchmark(
    "DataFrame",
    lambda: word_count_dataframe(FILEPATH).collect(),
    n_runs=N_RUNS
)
print(f"DataFrame: {df_bench['mean']:.3f}s ± {df_bench['std']:.3f}s")

speedup = rdd_bench['mean'] / df_bench['mean']
print(f"\n🚀 DataFrame is {speedup:.2f}x faster")

---
## Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Performance Comparison ---
ax1 = axes[0]
names = ["RDD", "DataFrame"]
means = [rdd_bench["mean"], df_bench["mean"]]
stds = [rdd_bench["std"], df_bench["std"]]
colors = ["#e74c3c", "#3498db"]

bars = ax1.bar(names, means, yerr=stds, capsize=8, color=colors, 
               edgecolor="black", linewidth=1.5)

ax1.set_ylabel("Execution Time (seconds)", fontsize=12)
ax1.set_title("Word Count Performance: RDD vs DataFrame", fontsize=14, fontweight="bold")

# Add value labels
for bar, mean, std in zip(bars, means, stds):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.05,
            f"{mean:.3f}s", ha="center", fontsize=12, fontweight="bold")

# Add speedup annotation
ax1.annotate(f"{speedup:.1f}x faster", 
            xy=(1, df_bench["mean"]), 
            xytext=(0.5, (rdd_bench["mean"] + df_bench["mean"])/2),
            fontsize=14, fontweight="bold", color="#27ae60",
            arrowprops=dict(arrowstyle="->", color="#27ae60", lw=2))

# --- Top Words ---
ax2 = axes[1]
top_words = df_result[:15]
words = [row["word"] for row in top_words]
counts = [row["count"] for row in top_words]

y_pos = np.arange(len(words))
ax2.barh(y_pos, counts, color="#3498db", edgecolor="black", linewidth=0.5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(words, fontsize=11)
ax2.invert_yaxis()
ax2.set_xlabel("Frequency", fontsize=12)
ax2.set_title("Top 15 Words in Shakespeare", fontsize=14, fontweight="bold")

for i, count in enumerate(counts):
    ax2.text(count + 200, i, f"{count:,}", va="center", fontsize=9)

plt.tight_layout()
plt.savefig("benchmark_results.png", dpi=150, bbox_inches="tight")
plt.show()

---
## Why DataFrames Are Faster

### 1. Catalyst Optimizer
- **Predicate pushdown**: Filters moved closer to data source
- **Column pruning**: Only reads columns actually needed
- **Constant folding**: Evaluates constant expressions at compile time
- **Join reordering**: Optimizes join order based on statistics

### 2. Tungsten Execution Engine
- **Off-heap memory**: Avoids JVM garbage collection overhead
- **Cache-aware computation**: Optimized for CPU cache hierarchy
- **Whole-stage code generation**: Compiles query stages to optimized bytecode

### 3. Efficient Data Representation
- RDDs: Java objects with full serialization overhead
- DataFrames: Binary row format, compact and efficient

---
## When to Use RDDs

RDDs still have valid use cases:

1. **Unstructured data** that doesn't fit a columnar schema
2. **Low-level transformations** not expressible in DataFrame API
3. **Fine-grained partitioning control** for specific performance tuning
4. **Legacy code** that predates DataFrame API
5. **Type safety in Scala** with compile-time checking

For most structured data processing: **prefer DataFrames**.

In [ ]:
# Cleanup
spark.stop()
print("Spark session stopped.")